In [3]:
%matplotlib inline
import numpy as np
import scipy as sp
import matplotlib as mpl
import matplotlib.cm as cm
import matplotlib.pyplot as plt
import pandas as pd
pd.set_option('display.width', 500)
pd.set_option('display.max_columns', 100)
pd.set_option('display.notebook_repr_html', True)
import seaborn as sns
sns.set_style("whitegrid")
sns.set_context("poster")
from sklearn.feature_extraction.text import CountVectorizer
import gensim
from sklearn.cross_validation import train_test_split
from collections import defaultdict
from pattern.en import parse
from pattern.en import pprint
from pattern.vector import stem, PORTER, LEMMA
from sklearn.feature_extraction import text 
stopwords=text.ENGLISH_STOP_WORDS
punctuation = list('.,;:!?()[]{}`''\"@#$^&*+-|=~_')



In [ ]:
documents = ["Human machine interface for lab abc computer applications",
"A survey of user opinion of computer system response time",
            "The EPS user interface management system",
        "System and human system engineering testing of EPS",
           "Relation of user perceived response time to error measurement",
            "The generation of random binary unordered trees",
             "The intersection graph of paths in trees",
             "Graph minors IV Widths of trees and well quasi ordering",
             "Graph minors A survey"]

vectorizer_t = CountVectorizer(min_df=1, stop_words='english')
X_t = vectorizer_t.fit_transform(documents)
corpus_gensim_t = gensim.matutils.Sparse2Corpus(X_t, documents_columns=False)

In [5]:
bike_masterdf = pd.read_csv('./bike_details.csv')
print "Number of samples in the database:", bike_masterdf.shape[0]
bike_masterdf.head(2)


tempdf = bike_masterdf.head(100).copy()
tempdf.to_json('./temp_json.json')

Number of samples in the database: 59619


In [ ]:
#SPark setup

conf = (pyspark.SparkConf()
    .setMaster('local')
    .setAppName('pyspark')
    .set("spark.executor.memory", "4g"))
sc = pyspark.SparkContext(conf=conf)
sqlsc=SQLContext(sc)



In [8]:
thetext="The bike was locked to a sign pole by the corner of the superstore. The lock was a combination lock and i had \
tied chained my helmet to the bike. The bike was of make 'GT' with shimano gears and had a custom seat"
parse(thetext, tokenize=True, lemmata=True).split()

[[[u'The', u'DT', u'B-NP', u'O', u'the'],
  [u'bike', u'NN', u'I-NP', u'O', u'bike'],
  [u'was', u'VBD', u'B-VP', u'O', u'be'],
  [u'locked', u'VBN', u'I-VP', u'O', u'lock'],
  [u'to', u'TO', u'O', u'O', u'to'],
  [u'a', u'DT', u'B-NP', u'O', u'a'],
  [u'sign', u'NN', u'I-NP', u'O', u'sign'],
  [u'pole', u'NN', u'I-NP', u'O', u'pole'],
  [u'by', u'IN', u'B-PP', u'B-PNP', u'by'],
  [u'the', u'DT', u'B-NP', u'I-PNP', u'the'],
  [u'corner', u'NN', u'I-NP', u'I-PNP', u'corner'],
  [u'of', u'IN', u'B-PP', u'B-PNP', u'of'],
  [u'the', u'DT', u'B-NP', u'I-PNP', u'the'],
  [u'superstore', u'NN', u'I-NP', u'I-PNP', u'superstore'],
  [u'.', u'.', u'O', u'O', u'.']],
 [[u'The', u'DT', u'B-NP', u'O', u'the'],
  [u'lock', u'NN', u'I-NP', u'O', u'lock'],
  [u'was', u'VBD', u'B-VP', u'O', u'be'],
  [u'a', u'DT', u'B-NP', u'O', u'a'],
  [u'combination', u'NN', u'I-NP', u'O', u'combination'],
  [u'lock', u'NN', u'I-NP', u'O', u'lock'],
  [u'and', u'CC', u'O', u'O', u'and'],
  [u'i', u'NN', u'B-NP', u'O

In [10]:
num_topics=range(1,11)
num_topics

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10]

In [6]:

a = np.arange(0,60,10)
a

array([ 0, 10, 20, 30, 40, 50])